# 오차역전파법

앞 장에서는 신경망 학습에 대해서 배웠다. 그때 신경망의 가중치 매개변수의 기울기(가중치 매개변수에 대한 손실 함수의 기울기)는 수치 미분을 사용해 구하는 방법을 사용했다. 수치 미분은 단순하고 구현하기도 쉽지만 계산 시간이 오래 걸린다는 게 단점이다. 딥러닝의 모델들은 층(Layer)를 깊게 쌓고, 또 방대한 양의 데이터를 사용하기 때문에, 계산 시간을 단축하는 것이 핵심이다. 따라서 이번 장에서는 가중치 매개변수의 기울기를 효과적으로 계산하는 **'오차역전파법'**에 대해 배운다.

## 5.1 계산 그래프

**계산 그래프**는 계산 과정을 그래프의 자료구조인 **노드(node)**와 **엣지(edge)**로 표현하는 방법이다.

문제 1 : 승리가 슈퍼에서 1개에 100원인 사과를 2개 샀다. 이때의 지불금액을 구하시오. 단, 소비세가 10% 부과된다.

![Imgur](https://i.imgur.com/gC8Rp0J.png)

문제 2: 승리가 슈퍼에서 개당 100원인 사과를 2개, 개당 150원인 귤을 3개 샀다. 소비세가 10% 부과될 때, 전체 지불 금액은?

![Imgur](https://i.imgur.com/utrpeCS.png)

계산 그래프의 특징은 **'국소적 계산'**을 전파함으로써 최종 결과를 얻는다는 점이다. 각 노드에서의 계산은 자신에게 주어진 input만 고려하고, 그 이전에 어떤 복잡한 계산이 있었는지는 고려하지 않는다. 단순한 계산만이 각 노드에서 수행되기 때문에, 이후에 **'역전파'**에서도 간단한 미분만 이루어져 시간을 단축시킬 수 있다. 

## 5.2 Chain rule

![Imgur](https://i.imgur.com/UdlyUdJ.png)

위의 그림과 같이 역전파의 계산은 상류에서 전달된 신호 ***E***에 노드의 국소적 미분(**'x에 대한 y의 미분'**)을 곱한 후 앞쪽 노드로 전달하게 된다. 

### Chain rule

합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

![Imgur](https://i.imgur.com/Fge55tH.png)

위의 식을 계산 그래프로 나타내보자.

![Imgur](https://i.imgur.com/n81qmBm.png)

Chain rule에 따라, 가장 왼쪽의 역전파의 계산은 분자와 분모들끼리 소거가 되어, 결국 **'x에 대한 z의 미분'**이 된다.

![Imgur](https://i.imgur.com/L2Eft6v.png)

## 5.3 역전파

### 덧셈 노드의 역전파

z = x + y

![Imgur](https://i.imgur.com/9ljp85A.png)

![Imgur](https://i.imgur.com/qTn7qIq.png)

덧셈 노드의 역전파는 상류에서 전해진 미분(여기서는 **'z에 대한 L의 미분'**)에 1을 곱하여 그대로 앞쪽 노드로 전달한다.

### 곱셈 노드의 역전파

z = x * y

![Imgur](https://i.imgur.com/QRCXoie.png)

![Imgur](https://i.imgur.com/nQ8OXOm.png)

곱셈 노드의 역전파는 상류에서 전해진 미분(여기서는 **'z에 대한 L의 미분'**)에 순전파 때의 입력 신호들을 **'서로 바꾼 값'**을 곱해서 앞쪽 노드로 전달한다. 순전파 때 **x**였다면 역전파에서는 **y**, 순전파 때 **y**였다면 역전파에서는 **x**를 상류에서 전해진 값과 곱하여 앞쪽 노드로 전달한다. (따라서 구현시에, 순전파의 입력 신호 **x**, **y**를 변수에 저장할 필요가 있다.)

## 5.4 단순한 계층 구현하기

### 곱셈 계층

In [4]:
class MulLayer:
    
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

![Imgur](https://i.imgur.com/zUiPa0W.png)

위의 그림처럼 사과 2개를 구입하는 경우의 역전파를 구해보자.

In [5]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward
dprice = 1 # 처음으로 전달되는 역전파의 값이 1인 경우 가정
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


### 덧셈 계층

In [6]:
class AddLayer:
    
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

![Imgur](https://i.imgur.com/CBAAj1g.png)

위의 그림처럼 사과 2개와 귤 3개를 사는 경우의 역전파를 구해보자.

In [7]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # 1단계
orange_price = mul_orange_layer.forward(orange, orange_num)  # 2단계
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # 3단계
price = mul_tax_layer.forward(all_price, tax)  # 4단계

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # 4단계
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # 3단계
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # 2단계
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # 1단계

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 5.5 활성화 함수 계층 구현하기

### ReLU 계층

ReLU 수식은 다음과 같다.

![Imgur](https://i.imgur.com/HQuwqtM.png)

ReLU 수식의 x에 대한 y의 미분은 다음과 같다.

![Imgur](https://i.imgur.com/3uKU35C.png)

순전파 때의 입력 x가 0보다 크면 역전파는 상류의 값을 그대로 앞쪽 노드로 전달한다. 반면 x가 0이하면, 신호를 전달하지 않는다. (0을 전달)

![Imgur](https://i.imgur.com/Y3abLB9.png)

In [8]:
class Relu:
    
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0) # self.mask는 원소의 값이 0이하인 경우의 원소의 위치를 True로, 0보다 큰 경우의 원소의 위치를 False로 저장.
        out = x.copy()
        out[self.mask] = 0 # True인 위치(원소의 값이 0이하)의 원소를 0으로 바꾸어준다.

        return out

    def backward(self, dout):
        dout[self.mask] = 0 # forward에서 만들어 둔, self.mask를 이용하는 것을 참고. 마찬가지로 True인 위치의 원소를 0으로 바꾸어준다.
        dx = dout

        return dx

### Sigmoid 계층

**Sigmoid** 수식은 다음과 같다.

![Imgur](https://i.imgur.com/2LhcTpm.png)

이를 계산 그래프로 나타내면 다음과 같다.

![Imgur](https://i.imgur.com/zKu16yW.png)

#### 1단계

나눗셈 노드, **y = 1 / x**을 미분하면 다음 식이 된다.

![Imgur](https://i.imgur.com/816ZQZX.png)

역전파 때는 상류에서 전달된 값에 **-y^2** (순전파의 출력을 제곱한 후 마이너스를 붙인 값)을 곱해서 앞쪽 노드로 전달한다.

![Imgur](https://i.imgur.com/zKu16yW.png)

#### 2단계

덧셈 노드, 상류에서 전달된 값을 그대로 앞쪽 노드로 전달한다.

![Imgur](https://i.imgur.com/WfHtclo.png)

#### 3단계

**exp** 노드, **exp** 함수를 미분한 결과는 다음과 같다.

![Imgur](https://i.imgur.com/5FrM5aC.png)

따라서 상류에서 전달된 값에 순전파 때의 출력 **exp(x) = y** 를 곱해서 앞쪽 노드로 전달한다.

![Imgur](https://i.imgur.com/6Nc1VuY.png)

#### 4단계

곱셈 노드, 상류에서 전달된 값에 순전파 때의 출력을 서로 바꿔서 곱해 앞쪽 노드로 전달한다.

![Imgur](https://i.imgur.com/UjjmXLN.png)

최종적으로 정리하면 **Sigmoid** 노드의 역전파 결과는 다음과 같다.

![Imgur](https://i.imgur.com/oVU3M1q.png)

위의 **Sigmoid** 노드의 역전파 결과는 다음과 같이 정리할 수 있다.

![Imgur](https://i.imgur.com/61bjygp.png)

따라서 정리한 **Sigmoid** 노드의 역전파 결과는 다음과 같이 간단하게 표현 할 수 있다.

![Imgur](https://i.imgur.com/qZnGEMN.png)

In [9]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class Sigmoid:
    
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## 5.6 Affine / Softmax 계층 구현하기

### Affine 계층

신경망의 순전파에서 한 노드에서의 가중치 신호의 총합을 계산할 때, 행렬의 곱을 사용한다.

![Imgur](https://i.imgur.com/AZzZvdp.png)

이러한 다차원 행렬의 역전파 결과는 계산하면 최종적으로 다음과 같다. (계산과정은 생략)

![Imgur](https://i.imgur.com/znEnTnG.png)

신경망에서는 입력 데이터로 **X** 하나만이 아니라, **N**개를 묶어 배치(batch)를 사용합니다. 따라서 입력 데이터가 **(1,n)**이 아닌, **(m,n)**차원을 가지게 된다.

![Imgur](https://i.imgur.com/sNvHvVh.png)

In [10]:
class Affine:
    
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0) # 편향에는 그대로의 값이 전달된다. 다만 편향과의 차원(1,N)을 맞추기 위해, np.sum(axis=0)을 이용.
        
        return dx

### Softmax-with-Loss 계층

**Softmax**계층은 입력값을 정규화하여 출력한다. 또한 **MNIST**의 경우에는 분류하는 클래스의 개수가 10개이므로, **Softmax**의 입력 차원은 10개가 된다.

**Softmax**계층과 교차 엔트로피 오차를 구하는 과정을 묶어 **Softmax-with-Loss**계층으로 구현한다.

![Imgur](https://i.imgur.com/UPTYfXt.png)

이를 간소화하면 다음과 같이 나타낼 수 있다.

![Imgur](https://i.imgur.com/eorxyxd.png)

위의 그림에서 역전파의 결과에 주목하면, **Softmax**계층의 역전파가 매우 깔끔한 결과를 보인다는 것이다. **Softmax**계층의 출력과 정답 레이블의 차를 앞쪽 노드로 전달한다. 예를 들어, 정답 레이블이 (0, 1, 0)이고, **Softmax**계층의 출력이 (0.3, 0.2, 0.5)라고 가정하면, 역전파의 결과로 (0.3, 0.2, 0.5) - (0, 1, 0) = (0.3, -0.8, 0.5)라는 오차를 전달한다. 이러한 계산 그래프를 통해, 매우 복잡해보이는 **Softmax-with-Loss**의 역전파를 간단하게 표현할 수 있다. 이러한 이유로 수치 미분이 아닌 **오차역전법**을 이번 장에서 배운 것이다.

In [44]:
def cross_entropy_error(y, t):
    
    # 데이터가 배치가 아닌, 하나가 들어오는 경우
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 one-hot vector인 경우, 정답 레이블의 인덱스만 반환하여 사용하면, 계산을 더 빠르게 할 수 있다.
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size 

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # overflow 방지
    return np.exp(x) / np.sum(np.exp(x))

class SoftmaxWithLoss:
    
    def __init__(self):
        self.loss = None # 손실함수의 값
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(one-hot encoding 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        
        if self.t.size == self.y.size: # 정답 레이블이 one-hot encoding 형태일 때
            dx = (self.y - self.t) / batch_size
            
        else: # 정답 레이블이 숫자로 들어오는 경우일 때
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1 # 정답 레이블이 있는 부분만 1을 빼준다.
            dx = dx / batch_size
        
        return dx

질문!)

마지막에 SoftmaxWithLoss 계층의 역전파 결과를 **batch_size**로 나누어 주는 이유가 무엇인지?

## 5.7 오차역전파법 구현하기

신경망 학습의 순서는 다음과 같다.

1. 미니배치 : 훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실함수 값을 줄이는 것이 목표.

2. 기울기 산출 : 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다. (**오차역전파** 사용)

3. 매개변수 갱신 : 가중치 매개변수를 기울기 방향으로 갱신한다. (**learning rate**에 따라 변화의 양을 조절할 수 있다.)

4. 반복

In [45]:
import numpy as np
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) # 가중치는 랜덤으로 초기화
        self.params['b1'] = np.zeros(hidden_size) # 편향은 0으로 초기화
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) # 가중치는 랜덤으로 초기화
        self.params['b2'] = np.zeros(output_size) # 편향은 0으로 초기화

        # 계층 생성
        self.layers = OrderedDict() # 순서가 있는 딕셔너리를 사용하여, 순전파 때는 추가한 순서대로 호출하고, 역전파 때는 거꾸로 호출한다.
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values(): # 각 계층을 순서대로 호출
            x = layer.forward(x) # 호출한 계층에 forward를 적용
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x) # 순전파의 결과
        return self.lastLayer.forward(y, t) # 순전파의 결과와 정답 레이블에 softmax-with-loss를 적용해 loss를 구한다.
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1 
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse() # 역전파는 거꾸로 계층을 호출한다.
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [46]:
from mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.12898333333333334 0.1275
0.9061833333333333 0.9105
0.9238166666666666 0.9258
0.9380166666666667 0.9373
0.9461833333333334 0.9448
0.9529833333333333 0.9509
0.9571333333333333 0.9543
0.9616833333333333 0.9582
0.9630833333333333 0.9589
0.9662333333333334 0.9615
0.9693833333333334 0.9622
0.9714166666666667 0.9646
0.9732666666666666 0.9666
0.97395 0.9663
0.9763 0.9672
0.9778 0.9688
0.97905 0.9695
